# Github Archive

Presentado por: Yosef Guevara Salamanca


## Ejercicio

Every public github event for the last few years stored as gzip compressed line-delimited JSON data. Watch out, the schema switches at the 2014-2015 transition.

[Download link](https://www.githubarchive.org/)

**Try the following:**

1. Use dask.bag to inspect the data
2. Drill down using functions like pluck and filter
3. Find who the most popular committers were in 2015

## 1. Use dask.bag to inspect the data

In [1]:
# Librerias necesarias

import dask.bag as db
import json
import numpy as np
import pandas as pd
import aiohttp
import requests

In [2]:
# Se crea un Dask bag llamado "git"

git = db.read_text('https://data.gharchive.org/2015-01-01-15.json.gz').map(json.loads)
git

dask.bag<loads, npartitions=1>

In [3]:
# Muestra tipo de objeyo que es la varaible git

type(git)

dask.bag.core.Bag

In [4]:
# Se toman 2 objetos dentro de la bolsa y se visualizan

git.take(2)

({'id': '2489651045',
  'type': 'CreateEvent',
  'actor': {'id': 665991,
   'login': 'petroav',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/petroav',
   'avatar_url': 'https://avatars.githubusercontent.com/u/665991?'},
  'repo': {'id': 28688495,
   'name': 'petroav/6.828',
   'url': 'https://api.github.com/repos/petroav/6.828'},
  'payload': {'ref': 'master',
   'ref_type': 'branch',
   'master_branch': 'master',
   'description': "Solution to homework and assignments from MIT's 6.828 (Operating Systems Engineering). Done in my spare time.",
   'pusher_type': 'user'},
  'public': True,
  'created_at': '2015-01-01T15:00:00Z'},
 {'id': '2489651051',
  'type': 'PushEvent',
  'actor': {'id': 3854017,
   'login': 'rspt',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/rspt',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3854017?'},
  'repo': {'id': 28671719,
   'name': 'rspt/rspt-theme',
   'url': 'https://api.github.com/repos/rspt/rspt-theme'},
 

In [5]:
# Se visualiza el tañamo de la bolsa

tamaño = git.count()
tamaño.compute()

11351

## 2. Drill down using functions like pluck and filter

Aplicación de la función de filtrado

In [6]:
# Muestra la cantidad de eventos Push generados en todo el DataSet

git.filter(lambda record: record['type'] == 'PushEvent').count().compute()

5815

In [7]:
# Muestra la cantidad de repositorios creados

git.filter(lambda record: record['type'] == 'CreateEvent').count().compute()

1471

In [8]:
# Muestra la cantidad de repositorios creados en la fecha 2015-01-01T15:00:01Z

git.filter(lambda record: record['created_at'] == '2015-01-01T15:00:01Z').count().compute()

2

In [9]:
# Muestra la cantidad de repositorios privados del dataset

git.filter(lambda record: record['public'] == True).count().compute()

11351

Aplicación de la función de pluck

In [10]:
# Crea una lista con 10 id de repositorios

list(git.pluck('id').take(10))

['2489651045',
 '2489651051',
 '2489651053',
 '2489651057',
 '2489651062',
 '2489651063',
 '2489651064',
 '2489651066',
 '2489651067',
 '2489651071']

In [11]:
# Muestra unos de los commits del repo 2489651051

git.filter(lambda record: record['id'] == '2489651051').pluck('payload').pluck('commits').take(1)

([{'sha': '6b089eb4a43f728f0a594388092f480f2ecacfcd',
   'author': {'email': '5c682c2d1ec4073e277f9ba9f4bdf07e5794dabe@rspt.ch',
    'name': 'rspt'},
   'message': 'Fix main header height on mobile',
   'distinct': True,
   'url': 'https://api.github.com/repos/rspt/rspt-theme/commits/6b089eb4a43f728f0a594388092f480f2ecacfcd'}],)

## 3. Find who the most popular committers were in 2015

In [12]:
# Se hace un llamado a la libreria RegEx para encontrar los patrones

import re

In [14]:
# Se verifica la estructura del usuario

str(git.pluck('repo').pluck('name').take(1))

"('petroav/6.828',)"

In [15]:
# Se crea una función para identificar el patrón de los nombres de usuarios

userRegEx = re.compile(r'(\w+\S*)+\/')

In [ ]:
mo = userRegEx.search(str(git.pluck('repo').pluck('name').take(1)))
print(mo.group())

In [18]:
# Se incializa una lista en blanco para los nombres de los usuarios
user_list =  str(git.pluck('repo').pluck('name').take(100))
user_list = userRegEx.findall(user_list)
user_list 

KeyboardInterrupt: 